<a href="https://colab.research.google.com/github/anisha98/DiffusionScore/blob/main/Execution_of_1_article(presented_in_paper).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/2123925574.zip

unzip:  cannot find or open /content/2123925574.zip, /content/2123925574.zip.zip or /content/2123925574.zip.ZIP.


In [1]:
import math
import queue
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# To read the data 

In [2]:
data = pd.read_csv("/content/2050230532.csv") # Read the csv file
data.head()

,Unnamed: 0,2620050178.0,2742205700.0,2050230532.0,2478582605.0,2024523156.0,1965585172.0,2755946900.0,2795767639.0,2518937691.0,2155777124.0,1969877606.0,2091066410.0,2894702700.0,2559684211.0,2776888374.0,1968197118.0
0,2.620050e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.742206e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.050231e+09,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,2.478583e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.024523e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
idx = data.index[data['Unnamed: 0'] ==2050230532] ## to obtain the source node index which is the name of the csv file
idx.tolist()
print(idx[0])

2


In [4]:
data = data.drop('Unnamed: 0',axis = 1) ## preprocess the data to make it into a numpy array

In [5]:
data = data.to_numpy()

In [7]:
N = len(data[0]) ##Total no of nodes in the matrix 
N

16

In [8]:
data

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,

# Adjacency Matrix

In [9]:
visited = [0]*N
stack = []
stack_alpha = [0]*N
adj = []
g = data
rows = g.shape[0]
cols = g.shape[1]
for x in range(0, rows):
    l = []
    for y in range(0, cols):
        if(g[x,y] == 1):
            l.append(y)
    adj.append(l)
print(adj)

[[], [], [3, 4, 9, 10, 15], [], [11], [4, 11], [], [], [0, 1], [5], [8, 12, 13], [4, 5], [], [7, 14], [], [6, 10, 11]]


In [10]:
d = [i for i in range(0,N)]
d.pop(0)
parent = {}
for j in d:
    parent[j] = []
    for i in range(len(adj)):
        if j in adj[i]:
          parent[j].append(i)
parent   

{1: [8],
 2: [],
 3: [2],
 4: [2, 5, 11],
 5: [9, 11],
 6: [15],
 7: [13],
 8: [10],
 9: [2],
 10: [2, 15],
 11: [4, 5, 15],
 12: [10],
 13: [10],
 14: [13],
 15: [2]}

# Alpha value generated hopwise to see how we can reach all the nodes in the network from every other node in the network.This is used for calculation of the closeness centrality.

In [11]:
alpha_matrix = []
def DFS(v,visited,stack,stack_alpha):
    visited[v] = 1
    stack.append(v)
    for i in adj[v]:
        if visited[i] == 0:
            stack_alpha[i] = stack_alpha[v] + 1.0
            DFS(i,visited,stack,stack_alpha)
    #print("Stack_alpha",stack_alpha)
    return stack_alpha

In [12]:
for i in range(N):
    initial_node = i
    stack = []
    stack_alpha = [0.0]*N
    visited = [0]*N
    stack_alpha[i] = 1.0
    p = DFS(i,visited,stack,stack_alpha)
    alpha_matrix.append(p)
for i in range(N):
    for j in range(N):
        if(alpha_matrix[i][j] > 0.0):
            alpha_matrix[i][j] -=1
for i in range(N):
    alpha_matrix[i][i] = 0.0
print("Alpha_matrix",alpha_matrix)

layers = alpha_matrix

('Alpha_matrix', [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.0, 3.0, 0.0, 1.0, 1.0, 3.0, 2.0, 3.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0], [2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# Closeness Matrix


In [13]:
c = np.zeros(shape=(N,1))
for i in range(N):
    sum_dist = 0
    for j in range(N):
        if(alpha_matrix[i][j] >0):
            sum_dist += 1
    c[i,0] = sum_dist
c
#c = np.array([[1.0],[1.0],[0.0]])

array([[ 0.],
       [ 0.],
       [15.],
       [ 0.],
       [ 2.],
       [ 2.],
       [ 0.],
       [ 0.],
       [ 2.],
       [ 3.],
       [ 7.],
       [ 2.],
       [ 0.],
       [ 2.],
       [ 0.],
       [12.]])

# Degree Matrix


In [14]:
H = np.zeros(shape=(N,N))
row = H.shape[0]
col = H.shape[1]
d = np.zeros(shape=(N,1)) #degree of the graph
din = np.zeros(shape=(N,1))
dout = np.zeros(shape=(N,1))
g = data
#count = -10
for y in range(0,row):
    p = g[y] == np.array([0]*N)
    equal = p.all()
    for x in range(0,col):
        dout[y,0] -= g[y,x] #outgoing nodes
        din[y,0] += g[x,y] #ingoing nodes   
    d[y,0] = din[y,0] + dout[y,0]
print(d)

[[ 1.]
 [ 1.]
 [-5.]
 [ 1.]
 [ 2.]
 [ 0.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 0.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-2.]]


In [15]:
for i in range(N):
    if(c[i,0] == 0):
        c[i,0] = 1
print(c)

[[ 1.]
 [ 1.]
 [15.]
 [ 1.]
 [ 2.]
 [ 2.]
 [ 1.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 7.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 1.]
 [12.]]



# Putting together the concept of degree,closeness centrality and distance of a node from the source node.We come up with a parameter(heat_alpha) we can feed into the heat kernel

In [16]:
alpha_matrix = []
def DFS(v,visited,stack,stack_alpha):
    visited[v] = 1
    stack.append(v)
    #print("Visited",v)
    #print("Stack",stack)
    for i in adj[v]:
        #print("Current",i)
        if visited[i] == 0:
            stack_alpha[i] = stack_alpha[v] -d[i,0]/N + c[i,0]/N
            DFS(i,visited,stack,stack_alpha)
    #print("Stack_alpha",stack_alpha)
    return stack_alpha

In [17]:
for i in range(N):
    initial_node = i
    stack = []
    stack_alpha = [0.0]*N
    visited = [0]*N
    stack_alpha[i] = 1.0
    p = DFS(i,visited,stack,stack_alpha)
    alpha_matrix.append(p)
#pl = [3,2,2,1,2,1]
# for i in range(N):
#     alpha_matrix[i][i] = -1.0
alpha_matrix

[[1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [1.6875,
  1.6875,
  1.0,
  1.0,
  1.0,
  1.1875,
  1.875,
  1.6875,
  1.6875,
  1.1875,
  1.5,
  1.0625,
  1.5,
  1.6875,
  1.6875,
  1.875],
 [0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.1875,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0625,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0625,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,


In [18]:
for i in range(N):
  alpha_matrix[i][i] = -1.0

# Heat kernal

In [19]:
H = np.zeros(shape=(N,N))
row = H.shape[0]
col = H.shape[1]
g = data
for x in range(0,N):
  for y in range(0,N):  
     if g[y,x] == 1 and alpha_matrix[y][x] != 0:
       H[x,y]  = 1/alpha_matrix[y][x] 
     else:
       H[x,y] = -alpha_matrix[y][x]/N
#print(H)#heat kernel

In [20]:
initial_node = idx[0]
arr = [0.0]*N
arr[initial_node] = 0.0758427 ## Initial heat in the source node
initial_heat = np.array(arr) #0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
heat = np.dot((np.exp(H)),initial_heat)
print(heat)

[0.06825104 0.06825104 0.08073413 0.20616183 0.20616183 0.07041756
 0.06745589 0.06825104 0.06825104 0.17604977 0.14772141 0.07096985
 0.06905556 0.06825104 0.06825104 0.12928184]


In [22]:
initial_heat

array([0.       , 0.       , 0.0758427, 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       ])

In [23]:
heat = np.dot((np.exp(H)),initial_heat)
heat

array([0.06825104, 0.06825104, 0.08073413, 0.20616183, 0.20616183,
       0.07041756, 0.06745589, 0.06825104, 0.06825104, 0.17604977,
       0.14772141, 0.07096985, 0.06905556, 0.06825104, 0.06825104,
       0.12928184])

In [24]:
#Needs to be copy pasted from above
heat = np.array([0.06825104, 0.06825104, 0.08073413, 0.20616183, 0.20616183,
       0.07041756, 0.06745589, 0.06825104, 0.06825104, 0.17604977,
       0.14772141, 0.07096985, 0.06905556, 0.06825104, 0.06825104,
       0.12928184])

In [25]:
heat1 = [0.0]*N
for i in range(len(heat)):
    print(i)
    heat1[i] = ( (heat[i] - heat.min()) / (heat.max() - heat.min()) ) * (2 - 1) + 1
    print(heat1[i])

0
1.0057326312052677
1
1.0057326312052677
2
1.0957294258630885
3
2.0
4
2.0
5
1.0213521497348996
6
1.0
7
1.0057326312052677
8
1.0057326312052677
9
1.7829072064253342
10
1.5786739918996981
11
1.025333882600846
12
1.0115328153934864
13
1.0057326312052677
14
1.0057326312052677
15
1.4457339750554303


In [26]:
for i in range(len(heat1)):
    heat1[i] = (heat1[i]/2.0) * arr[idx[0]]
print(heat1)
for i in range(len(heat)):
    heat[i] = heat1[i]

[0.03813873911435588, 0.03813873911435588, 0.041551539063453234, 0.0758427, 0.0758427, 0.03873105234334954, 0.03792135, 0.03813873911435588, 0.03813873911435588, 0.06761024819237735, 0.05986544898272562, 0.03888204502896559, 0.038358689929021784, 0.03813873911435588, 0.03813873911435588, 0.054824184074968245]


In [27]:
print(heat)

[0.03813874 0.03813874 0.04155154 0.0758427  0.0758427  0.03873105
 0.03792135 0.03813874 0.03813874 0.06761025 0.05986545 0.03888205
 0.03835869 0.03813874 0.03813874 0.05482418]
